In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import geopandas as gpd
import pandas as pd
import folium
import json
from loguru import logger

In [ ]:
sys.path.append("../")  # include parent directory
from src.config_utils import build_kwargs_from_config

# Visualize Predictions

This notebook creates a folium map of the model predicted polygons along with the ground truth annotations

## Input
- Predicted Polygons (gpkg file)
- Annotated Polygons (gpkg file)

## Output
- Interactive Folium map (html file)

## Set Input Parameters

In [ ]:
DATA_PATH = Path("../data")
CONFIG_PATH = Path("../config")

CONFIG_FPATH = CONFIG_PATH / "pond_config.yaml"

ANNOTATIONS_GPKG_FPATH = DATA_PATH / "training_polygons.gpkg"
PLANET_CONFIG_FPATH = CONFIG_PATH / "secrets" / "planet_config.json"

# we need to simplify the polygons for the folium map for performance
SIMPLIFY_M = 4

# Select country (to speed up map)
# COUNTRY = 'Indonesia'
COUNTRY = "Philippines"

In [ ]:
kwargs_dict = build_kwargs_from_config(DATA_PATH, CONFIG_FPATH)

In [ ]:
MISC_KWARGS = kwargs_dict["misc_kwargs"]
PARENT_DIR = MISC_KWARGS["parent_dir"]

In [ ]:
PRED_POLYGONS_FPATH = DATA_PATH / PARENT_DIR / "pred_polygons.gpkg"
FOLIUM_MAP_FPATH = (
    DATA_PATH / PARENT_DIR / f"{COUNTRY}_predicted_polygons_map_folium.html"
)
PRED_POLYGONS_FPATH, FOLIUM_MAP_FPATH

## Load Planet Credentials for NICFI basemap

In [ ]:
with open(str(PLANET_CONFIG_FPATH)) as file:
    planet_config = json.load(file)
planet_xyz_url = "https://tiles3.planet.com/basemaps/v1/planet-tiles/planet_medres_visual_2022-07_mosaic/gmap/{z}/{x}/{y}.png?api_key="
planet_xyz_url = planet_xyz_url + planet_config["PLANET_API_KEY"]

## Load the annotation polygons and the model prediction polygons

In [ ]:
annotations = gpd.read_file(ANNOTATIONS_GPKG_FPATH)
usecols = ["id", "label", "geometry"]

# drop the unsure class
bool_mask = annotations["label"] == "unsure"
logger.info(f"Removing {bool_mask.sum():,} rows with the unsure label")
annotations = annotations.loc[~bool_mask, usecols]

annotations.head()

In [ ]:
pred_polygons = gpd.read_file(PRED_POLYGONS_FPATH)
pred_polygons.head()

### Get the country admin bounds and join with the aquaculture polygons

In [ ]:
admin_bounds = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
usecols = ["name", "geometry"]
admin_bounds = admin_bounds.loc[admin_bounds["name"] == COUNTRY, usecols]
admin_bounds.head()

In [ ]:
country_pred_polygons = gpd.sjoin(
    left_df=pred_polygons,
    right_df=admin_bounds,
    how="inner",
    predicate="intersects",
)
country_pred_polygons["geometry"] = country_pred_polygons["geometry"].simplify(
    SIMPLIFY_M
)
country_pred_polygons.head(2)

In [ ]:
country_annotations = gpd.sjoin(
    left_df=annotations,
    right_df=admin_bounds,
    how="inner",
    predicate="intersects",
)

country_annotations["geometry"] = country_annotations["geometry"].simplify(SIMPLIFY_M)
country_annotations.head(2)

### Make folium map

In [ ]:
m = folium.Map(tiles=None)

folium.raster_layers.TileLayer(tiles=planet_xyz_url, attr="NICFI", name="NICFI").add_to(
    m
)

m = country_annotations.explore(
    m=m,
    column="label",
    categorical=True,
    name="Ground Truth Annotations",
    cmap=[
        "#d62728",
        "#2ca02c",
        "#1f77b4",
    ],  # in alphabetical order: abandoned, extensive, intensive
    tooltip=False,
    show=False,
)

m = country_pred_polygons.explore(
    m=m,
    column="label",
    categorical=True,
    name="Model Predictions",
    cmap=["#d62728", "#2ca02c", "#1f77b4"],
    tooltip=False,
)

# Fit map to bounds
minx, miny, maxx, maxy = admin_bounds.total_bounds
m.fit_bounds([[miny, minx], [maxy, maxx]])

folium.LayerControl().add_to(m)

### Save Map

In [ ]:
m.save(FOLIUM_MAP_FPATH)